In [ ]:
#由于官方提供了不参与最终评价的目标数据，我们使用迁移学习来利用这些数据

In [ ]:
import tensorflow as tf 
import tensorflow_addons as tfa
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K

In [ ]:
def build(shape=None,out_cols=206):
    model = tf.keras.models.Sequential([
                L.InputLayer(input_shape=shape),
                L.BatchNormalization(),
                L.Dropout(0.5),
                tfa.layers.WeightNormalization(L.Dense(1500,kernel_initializer="he_normal")),
                
                L.BatchNormalization(),
                L.Activation(tf.nn.leaky_relu),
                L.Dropout(0.3),
                tfa.layers.WeightNormalization(L.Dense(1250,kernel_initializer="he_normal")),
                
                L.BatchNormalization(),
                L.Activation(tf.nn.leaky_relu),
                L.Dropout(0.3),
                tfa.layers.WeightNormalization(L.Dense(1000,kernel_initializer="he_normal")),
        
                L.BatchNormalization(),
                L.Activation(tf.nn.leaky_relu),
                L.Dropout(0.25),
                tfa.layers.WeightNormalization(L.Dense(750,kernel_initializer="he_normal")),
        
                L.BatchNormalization(),
                L.Activation(tf.nn.leaky_relu),
                L.Dropout(0.25),
                tfa.layers.WeightNormalization(L.Dense(out_cols,activation="sigmoid",kernel_initializer="he_normal"))
            ])
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.001)
                  ,optimizer = tfa.optimizers.AdamW(lr=0.001,weight_decay=1e-4),
                  metrics = ["binary_crossentropy"])
    
    return model

def metric(y_true,y_predicted):

    metrics=[]
    for col in range(y_true.shape[1]):
        metrics.append(log_loss(y_true[:,col],y_predicted[:,col],labels=[0,1]))

    return np.mean(metrics)

def transfer_weight(model_source,model_dest):
    for i in range(len(model_source.layers[:-1])):
        model_dest.layers[i].set_weights(model_source.layers[i].get_weights())
    return model_dest

In [ ]:
#将待评价目标与不评价目标一起放入训练
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')

In [ ]:
target_cols = [x for x in train_targets_scored.columns if x != 'sig_id']
aux_target_cols = [x for x in train_targets_nonscored.columns if x != 'sig_id']
all_target_cols = target_cols + aux_target_cols

num_targets = len(target_cols)
num_aux_targets = len(aux_target_cols)
num_all_targets = len(all_target_cols)

In [ ]:
X_train, X_val,Y_train,Y_val = train_test_split(train,train[all_target_cols],test_size=0.2,random_state=101)

In [ ]:
model = build((X_train.shape[1],),num_all_targets)

In [ ]:
save_weight = ModelCheckpoint('model.learned.hdf5',save_best_only=True,save_weights_only=False,monitor = 'val_loss',mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
early = EarlyStopping(monitor='val_loss',patience=5,mode='min')

model.fit(X_train,Y_train,
         epochs=50,
         batch_size=128,
         validation_data = (X_val,Y_val),
         callbacks=[save_weight,reduce_lr_loss,early])

model.load_weights('model.learned.hdf5')

In [ ]:
mskf = MultilabelStratifiedKFold(n_splits = 7,shuffle=True)
seeds = [42,58,132]
histories = []
scores = []

for seed in seeds:
    seedAll(seed_value=seed)
    print(f"Training seed {seed}")
    print('='*50)
    
    for idx,(tr_,val_) in enumerate(mskf.split(train,train_targets)):
        print(f'\nFold {idx}')
        print('-'*50)
        
        K.clear_session()
        X_train,X_val,Y_train,Y_val = train.iloc[tr_,:],train.iloc[val_,:],train[num_targets].iloc[tr_,:],train[num_targets].iloc[val_,:]
        
        path = f'model.{seed}_{idx}.hdf5'
        save_weight = ModelCheckpoint(path,save_best_only=True,save_weights_only=False,monitor = 'val_loss',mode='min')
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        early = EarlyStopping(monitor='val_loss',patience=5,mode='min')
        
        model_fin = build(shape=(X_train.shape[1],))
        model_fin = transfer_weight(model,model_fin)
        
        for layer in model_fin.layers:
            layer.trainable=True
            
        history = model_fin.fit(X_train.values,Y_train,
                 batch_size=128,
                 epochs=50,
                 validation_data=(X_val,Y_val),
                 callbacks=[early,save_weight,reduce_lr_loss]
                 )
        histories.append(history)
        model_fin= tf.keras.models.load_model(path, custom_objects={'leaky_relu': tf.nn.leaky_relu})
        
        val_pred = model_fin.predict(X_val)
        score = metric(Y_val.values,val_pred)
        scores.append(score)
        
        print(f"Validation Score: {score}")
        pred = model_fin.predict(test.values)
        
        ss.loc[:,train_targets.columns]+= pred